<a href="https://colab.research.google.com/github/jeffvestal/elastic_jupyter_notebooks/blob/main/load_embedding_model_from_hf_to_elastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this code to load a NLP model from Hugging Face for use inside Elastic's elasticsearch. 

You can set up a [free trial elasticsearch Deployment in Elastic Cloud](https://cloud.elastic.co/registration) and run the below code in [Google's Colab](https://colab.research.google.com) for free.

Requires Elastic version 8.0+ with a platinum or enterprise license (or trial license)

Example here is loading a [Zero Shot model](https://huggingface.co/typeform/distilbert-base-uncased-mnli)

[Elastic NLP Model Support Docs](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html) 

[Code summarized from the eland docs](https://github.com/elastic/eland)

Disclaimer: presented as is with no guarantee.

# Install eland and *elasticsearch*

In [ ]:
pip install eland

In [ ]:
pip install elasticsearch

In [ ]:
pip install transformers

In [ ]:
pip install sentence_transformers

In [ ]:
pip install torch==1.11

In [ ]:
import getpass
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient


# Set Connection and Auth

In [ ]:
es_url = getpass.getpass('Enter elasticsearch endpoint:  ') # endpoint https://<esurl>:<port>
es_user = getpass.getpass('Enter username:  ') # username
es_pass = getpass.getpass('Enter password:  ') # password

# Connect to Elastic and Load a Hugging Face Model

In [ ]:
es = Elasticsearch(es_url, basic_auth=(es_user,es_pass))

[Supported `task_type` values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [ ]:
# Download a Hugging Face Zero Shot model directly from the model hub

# https://huggingface.co/typeform/distilbert-base-uncased-mnli
#tm = TransformerModel("sentence-transformers/all-MiniLM-L12-v2", "text_embedding")
tm = TransformerModel("distilbert-base-cased-distilled-squad", "question_answering")

In [ ]:
# Export the model in a TorchScrpt representation which Elasticsearch uses
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
# model_path, config_path, vocab_path = tm.save(tmp_path) #pre 8.2.0
model_path, config, vocab_path = tm.save(tmp_path)

In [ ]:
# Import model into Elasticsearch
ptm = PyTorchModel(es, tm.elasticsearch_model_id())
# ptm.import_model(model_path, config_path, vocab_path) # pre 8.2.0
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 

# Deploy the model

In [ ]:
# List models in elasticsearch
m = MlClient.get_trained_models(es, )
m.body

In [ ]:
# Deploy the model

#  Model is listed -> 'model_id': 'typeform__distilbert-base-uncased-mnli'
model_id='distilbert-base-cased-distilled-squad'

# start trained model deployment
s = MlClient.start_trained_model_deployment(es, model_id=model_id)
s.body

# You can see model state in Kibana -> Machine Learning -> Model Management -> Trained Models

# Zero Shot Time!

In [ ]:
# future reference do not use yet
#z = MlClient.infer_trained_model_deployment(es, model_id =model_id, docs=docs, )

In [ ]:
# Using requests until MlClient.infer_trained_model_deployment is updated to accept inference extra configs
import requests
from requests.auth import HTTPBasicAuth
import urllib.parse

endpoint = '_ml/trained_models/%s/deployment/_infer' % model_id
url = urllib.parse.urljoin(es_url, endpoint)

body = {
  "docs": [
    {
      "text_field": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
    }
  ],
  "inference_config": {
    "zero_shot_classification": {
      "labels": [
        "mobile",
        "website",
        "billing",
        "account access"
      ],
      "multi_label": True
    }
  }
}

resp = requests.post(url, auth=HTTPBasicAuth(es_user, es_pass), json=body)
r = resp.json()
print('Predicted value is: %s with a probability of %0.2f%%' % (r['predicted_value'], r['prediction_probability'] * 100))
print('=-=-=-=')
print('Full Probability output:')
for c in r['top_classes']:
    print ('%s probability of %0.5f%%' % (c['class_name'], c['class_probability'] * 100))

In [ ]:
# Just to see the full doc
resp.json()